# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df

In [ ]:
df = pd.read_csv ('insurance.csv', sep = ",")

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную

In [ ]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
X = df [['age', 'sex', 'bmi', 'children', 'smoker',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest']]

y = df ['charges']

In [ ]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.15)

In [ ]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
678,56,1,36.100,3,0,0,0,0,1
609,30,1,37.800,2,1,0,0,0,1
574,57,0,34.295,2,0,1,0,0,0
307,30,0,33.330,1,0,0,0,1,0
1017,26,0,34.200,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
665,43,1,38.060,2,1,0,0,1,0
679,49,0,23.180,2,0,0,1,0,0
625,29,0,26.030,0,0,0,1,0,0
258,51,1,24.415,4,0,0,1,0,0


In [ ]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
980,54,1,25.460,1,0,1,0,0,0
275,47,0,26.600,2,0,1,0,0,0
28,23,1,17.385,1,0,0,1,0,0
100,41,0,31.600,0,0,0,0,0,1
11,62,0,26.290,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...
296,19,1,27.700,0,1,0,0,0,1
264,53,0,38.060,3,0,0,0,1,0
1098,52,0,30.875,0,0,1,0,0,0
136,19,1,34.100,0,0,0,0,0,1


In [ ]:
y_train

678     12363.54700
609     39241.44200
574     13224.05705
307      4151.02870
1017     3987.92600
           ...     
665     42560.43040
679     10156.78320
625      3736.46470
258     11520.09985
536      5972.37800
Name: charges, Length: 1137, dtype: float64

In [ ]:
y_test

980     25517.11363
275      9715.84100
28       2775.19215
100      6186.12700
11      27808.72510
           ...     
296     16297.84600
264     20462.99766
1098    23045.56616
136      1261.44200
974      2322.62180
Name: charges, Length: 201, dtype: float64

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель

In [ ]:
lr = LinearRegression ()

In [ ]:
lr.fit (X_train, y_train)


LinearRegression()

In [ ]:
lr.coef_

array([  265.398119  ,  -266.50715227,   318.8811636 ,   497.75141329,
       23502.1872501 ,   322.75292749,   399.71196068,  -376.96215898,
        -345.50272919])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание
lr.predict (X_test)

array([ 1.07422287e+04,  1.00122249e+04,  1.68806148e+01,  8.35048355e+03,
        3.57013129e+04,  2.83727861e+04,  1.29119554e+03,  2.95342253e+04,
        1.29273500e+04,  5.93195214e+03,  6.95406458e+03,  5.22814298e+03,
        1.64647475e+04,  7.68886055e+03,  1.14658031e+04,  4.83607054e+03,
        1.62195707e+04,  1.10998328e+04,  1.17369752e+04,  1.78391222e+03,
        1.44010568e+04,  2.99506874e+04,  4.63630395e+03,  1.63201583e+04,
        3.11969492e+04,  5.47429409e+03,  3.94428652e+04,  2.65946617e+04,
        7.28386012e+03,  3.01662444e+03,  1.29897045e+04, -5.28918621e+01,
        3.41121470e+03,  3.88899349e+03,  2.91660030e+04,  1.13781930e+04,
        1.21698932e+04,  1.33935846e+04,  2.64715608e+04,  1.03236098e+04,
        2.95347688e+04,  1.43584643e+04,  2.48171625e+03,  4.18746770e+03,
        3.66847572e+04,  9.99331387e+03,  1.34877853e+04,  5.44019896e+03,
        9.79261583e+03,  1.16295200e+04,  1.08069685e+04, -1.51216956e+03,
        2.84231102e+03,  

In [ ]:
pd.DataFrame([y_test.values, lr.predict(X_test)]).T

,0,1
0,25517.11363,10742.228667
1,9715.84100,10012.224926
2,2775.19215,16.880615
3,6186.12700,8350.483547
4,27808.72510,35701.312887
...,...,...
196,16297.84600,24503.768488
197,20462.99766,15057.028102
198,23045.56616,11706.929669
199,1261.44200,3042.420685


In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки
mean_squared_error(y_test, lr.predict(X_test))

41869422.978540495

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

35665542.832428016

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([26466.63668036])